In [ ]:
#Code to hide cells if needed
%%html
<style>
div.input {
    display:none;
}
</style>

In [ ]:
#%matplotlib notebook
#%pylab inline
import pandas as pd
import matplotlib.pyplot as plt
from ipywidgets import *
from IPython.display import display
#from IPython.html import widgets
import datetime
import pandas.io.data as web
from IPython.display import clear_output
plt.style.use('ggplot')
%matplotlib notebook

In [ ]:
#ENTER DATABASE SOURCE HERE AS DF

df = pd.ExcelFile('C:/users/******/documents/studies/BflyDBMacro2.xlsm', sheet = 'Data2')
df2 = df.parse('Data2')

In [ ]:
#Plotly Settings - Enter Plotly Login information. If None there is also code to use Pandas plotting capabilities below
import plotly
plotly.tools.set_credentials_file(username='*****', api_key='*****')
import plotly.plotly as py
from plotly.tools import FigureFactory as FF
from datetime import datetime
import cufflinks as cf
from plotly.offline import download_plotlyjs, init_notebook_mode, iplot

plotly.offline.init_notebook_mode() 

In [ ]:
#Build Dashboard GUI
exp1 = widgets.Text(description = "Expiration YYYYMMDD", width = 100)
text1 = widgets.Text(description = "Strike1", width = 100)
text2 = widgets.Text(description = "Strike2", width = 100)
text3 = widgets.Text(description = "Strike3", width = 100)
display(exp1)
display(text1,text2,text3)
button = widgets.Button(description='Plot Butterfly')
display(button)


def on_button_clicked(b):
    clear_output()
    
    #Existing DB has prices as strings and need to be converted to numeric
    df2['MarkP'] = pd.to_numeric(df2['MarkP'], errors='coerce')
    df2['Strike'] = pd.to_numeric(df2['Strike'], errors='coerce')
    
    #Pandas Pivot Table feature to organize the data. 
    table = pd.pivot_table(df2, index=["Strike", "Exp"],values="MarkP", 
                          columns='Date')
    #Save user input as variables
    str1 = int(text1.value)
    str2 = int(text2.value)
    str3 = int(text3.value)
    expiry = str(exp1.value)
    expiry = datetime.strptime(expiry,'%Y%m%d').strftime('%Y-%m-%d')
    
    #Query Pivot Table for selected expiry and dates
    table2 = table.query('Exp==@expiry')
    strike1 = table2.query('Strike==@str1').transpose().dropna()
    strike2 = table2.query('Strike==@str2').transpose().dropna()
    strike3 = table2.query('Strike==@str3').transpose().dropna()

    #Calculate butterfly prices. 
    bfly = pd.concat([strike1,strike2,strike3], axis=1)
    bfly = bfly.dropna()
    bfly2 = bfly[str1]+bfly[str3]-2*bfly[str2]
    bfly2.index = pd.to_datetime(bfly2.index, format="%Y%m%d")
    
    #Download SPX data with similar dates
    start = bfly2.index[1]
    end = bfly2.index[len(bfly2.index)-1]
    spx = web.DataReader("^GSPC",'yahoo',start, end)
    spx = spx.rename(columns = {'Adj Close':'SPX'})

    print("Butterfly: ", str1,"/",str2,"/",str3)
    
    #Plotly Charts
    bfly2.rename(columns={expiry:'Butterfly'},inplace=True)
    fig = FF.create_candlestick(spx.Open, spx.High, spx.Low, spx.Close, dates=spx.index, name = 'SPX')
    fig2 = bfly2.iplot(secondary_y = ['Butterfly'], asFigure=True)
    fig2['data'].extend(fig['data']) 
    iplot(fig2)

    
    #Pandas Plots if Needed
    #pylab.rcParams['figure.figsize'] = (20, 6)
    #bfly2.plot()
    #spx['SPX'].plot(secondary_y=True, legend=True)
    #plt.show()   

button.on_click(on_button_clicked)

